# QUICKSTART ([source](https://python.langchain.com/docs/use_cases/question_answering/quickstart))

LOAD THE DOCUMENTS

In [2]:
!pip3 install pypdf

In [3]:
from os import getenv
from dotenv import load_dotenv

load_dotenv(".env")

True

In [4]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("docs/caperucitaroja.pdf")
docs = loader.load()
pages = loader.load_and_split()
len(pages)

34

SPLIT DOCUMENT INTO CHUNCKS FOR EMBEDDING AND VECTOR STORAGE

"In this case we’ll split our documents into chunks of 1000 characters with 200 characters of overlap between chunks. The overlap helps mitigate the possibility of separating a statement from important context related to it. We use the RecursiveCharacterTextSplitter, which will recursively split the document using common separators like new lines until each chunk is the appropriate size. This is the recommended text splitter for generic text use cases." [source](https://python.langchain.com/docs/use_cases/question_answering/quickstart#indexing-split)

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    # add_start_index=True
)
all_splits = text_splitter.split_documents(docs)
len(all_splits)

34

STORE

"We need to index our chunks so we can search over them at runtime. The most common way to do this is to embed the contents of each document split and insert these embeddings into a vector database (or vector store). When we want to search over our splits, we take a text search query, embed it, and perform some sort of “similarity” search to identify the stored splits with the most similar embeddings to our query embedding. The simplest similarity measure is cosine similarity — we measure the cosine of the angle between each pair of embeddings (which are high dimensional vectors)." [source](https://python.langchain.com/docs/use_cases/question_answering/quickstart#indexing-store)

In [6]:
from langchain_community.vectorstores import (
    Chroma,
)  # Options: https://python.langchain.com/docs/integrations/vectorstores

# from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import (
    GPT4AllEmbeddings,
)  # Replaces the OpenAI option

vectorstore = Chroma.from_documents(documents=all_splits, embedding=GPT4AllEmbeddings())

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [7]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
retrieved_docs = retriever.invoke("Como se llama la protagonista?")

In [8]:
len(retrieved_docs)

6

In [9]:
print(retrieved_docs[0].page_content)

La abuela vivía en el bosque, a media hora de camino desde el 
pueblo. Apenas Caperucita Roja entró en el bosque, salió a su 
encuentro un lobo. Nunca antes la niña había visto a un lobo y 
desconocía lo peligroso que es ese animal. 
El lobo, con su voz más amistosa, le dijo:
– ¡Buenos días, dulce pequeña! ¿Cómo te llamas?
– ¡Buenos días! Me llaman Caperucita Roja.
– ¿A dónde vas tan temprano?
– A ver a mi abuelita.
Caperucita roja.indd   6 17/09/2014   11:42:00


GENERATE

"Let’s put it all together into a chain that takes a question, retrieves relevant documents, constructs a prompt, passes that to a model, and parses the output." [source](https://python.langchain.com/docs/use_cases/question_answering/quickstart#retrieval-and-generation-generate)

In [10]:
from langchain_community.llms import GPT4All

llm = GPT4All(
    model="models/mistral-7b-openorca.gguf2.Q4_0.gguf",  # https://gpt4all.io/models/gguf/mistral-7b-openorca.gguf2.Q4_0.gguf
    max_tokens=2048,
)

Original prompt ([source](https://python.langchain.com/docs/use_cases/question_answering/quickstart#retrieval-and-generation-generate))

In [11]:
# from langchain import hub

# prompt = hub.pull("rlm/rag-prompt")

New prompt ([source](https://python.langchain.com/docs/use_cases/question_answering/quickstart#retrieval-and-generation-generate))

In [12]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Eres un asistente para responder preguntas. 
    Utiliza los siguientes fragmentos de contexto recuperado para responder la pregunta. 
    Si no conoces la respuesta, simplemente di que no la sabes. 
    Usa máximo tres oraciones y mantén la respuesta concisa.
    Pregunta: {question}
    Contexto: {context}
    Respuesta:
    """
)
prompt_template.format(question="pregunta ejemplo", context="contexto ejemplo")

'\n    Eres un asistente para responder preguntas. \n    Utiliza los siguientes fragmentos de contexto recuperado para responder la pregunta. \n    Si no conoces la respuesta, simplemente di que no la sabes. \n    Usa máximo tres oraciones y mantén la respuesta concisa.\n    Pregunta: pregunta ejemplo\n    Contexto: contexto ejemplo\n    Respuesta:\n    '

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"question": RunnablePassthrough(), "context": retriever | format_docs}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [14]:
rag_chain.invoke("A quien iba a visitar caperucita roja?")

' Caperucita Roja iba a visitar a su abuela enferma y débil.'

In [15]:
for chunk in rag_chain.stream("A quien iba a visitar caperucita roja?"):
    print(chunk, end="", flush=True)

 Caperucita Roja iba a visitar a su abuela enferma y débil.

In [16]:
rag_chain.invoke(
    "Que preguntas le hizo caperucita roja al lobo disfrazado de su abuela?"
)

' La pregunta que le hizo el lobo disfrazado de su abuela a Caperucita Roja fue "¿Cómo te llamas?" y luego preguntó "¿A dónde vas tan temprano?".'

In [17]:
for chunk in rag_chain.stream(
    "Que preguntas le hizo caperucita roja al lobo disfrazado de su abuela?"
):
    print(chunk, end="", flush=True)

 La pregunta que le hizo el lobo disfrazado de su abuela a Caperucita Roja fue "¿Cómo te llamas?" y luego preguntó "¿A dónde vas tan temprano?".

In [18]:
rag_chain.invoke(
    "El lobo se comio a la abuela de caperucita roja?"
)

' Sí, el lobo se comió a la abuela de Caperucita Roja en la versión original del cuento.'

In [19]:
rag_chain.invoke(
    "Que pasa con el lobo en esta version del cuento?"
)

' En esta versión del cuento, el lobo intenta engañar y comer a Caperucita Roja pero es desmantelado por la niña y su abuela. El lobo se mete en la casa de la abuela para comérsela, pero al final es derrotado y no vuelve a aparecer.'

# RETURNING SOURCES ([source](https://python.langchain.com/docs/use_cases/question_answering/sources))

ADDING SOURCES

With LCEL it's easy to return the retrieved documents:

In [20]:
from langchain_core.runnables import RunnableParallel


rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt_template
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"question": RunnablePassthrough(), "context": retriever}
).assign(answer=rag_chain_from_docs)

In [21]:
rag_chain_with_source.invoke(
    "Que pasa con el lobo en esta version del cuento?"
)

{'question': 'Que pasa con el lobo en esta version del cuento?',
 'context': [Document(page_content='El lobo giró el picaporte. La puerta se abrió. Sin pronunciar \npalabra, fue directamente a la cama donde yacía la abuela y se \nla tragó de un solo bocado. Entonces, se puso sus ropas, se calzó \nsu cofia, cerró las cortinas y se metió en la cama.\nCaperucita roja.indd   14 17/09/2014   11:42:26', metadata={'page': 14, 'source': 'docs/caperucitaroja.pdf'}),
  Document(page_content='El lobo, después de haber saciado su apetito, se metió de nuevo \nen la cama y se durmió. \nUn rato después, un cazador pasó por delante de la casa y oyó \nlos ronquidos. Se preocupó... \n“La abuela ronca pero nunca tan fuerte. Miraré, no sea que le \npase algo”.\nY entró en la alcoba.\nCaperucita roja.indd   20 17/09/2014   11:42:44', metadata={'page': 20, 'source': 'docs/caperucitaroja.pdf'}),
  Document(page_content='El lobo pensó: “Esa joven y delicada niña será un suculento \nbocado. Sabrá mucho mejor q

# ADD CHAT HISTORY ([source](https://python.langchain.com/docs/use_cases/question_answering/chat_history))

Allows the user to have a back-and-forth conversation.

"In this guide we focus on adding logic for incorporating historical messages, and NOT on chat history management. Chat history management is [covered here](https://python.langchain.com/docs/expression_language/how_to/message_history)."

We need to update two things about our existing app:
- **Prompt:** Add support to historical messages as an input.
- **Contextualizing questions:** Add a sub-chain that takes the latest user question and reformulates it in the context of the chat history. This is needed in case the latest question references some context from past messages. For example, if a user asks a follow-up question like “Can you elaborate on the second point?”, this cannot be understood without the context of the previous message. Therefore we can’t effectively perform retrieval with a question like this.

## Contextualizing the question ([source](https://python.langchain.com/docs/use_cases/question_answering/chat_history#contextualizing-the-question))

In [23]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Se proporciona un historial de chat y la última pregunta \
    del usuario, la cual podría referirse al contexto del historial. Formula una pregunta independiente \
    que se pueda entender sin el historial. NO respondas la pregunta, solo reformúlala si es necesario, \
    de lo contrario, devuélvela tal cual.
"""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
contextualize_q_chain = contextualize_q_prompt | llm | StrOutputParser()

Using this chain we can ask follow-up questions that reference past messages and have them reformulated into standalone questions:

In [24]:
from langchain_core.messages import AIMessage, HumanMessage

contextualize_q_chain.invoke(
    {
        "chat_history": [
            HumanMessage(content="Que pasa con el lobo en esta version del cuento?"),
            AIMessage(content="En esta versión del cuento, el lobo intenta engañar y comer a Caperucita Roja pero es desmantelado por la niña y su abuela. El lobo se mete en la casa de la abuela para comérsela, pero al final es derrotado y no vuelve a aparecer."),
        ],
        "question": "A que te refieres con desmantelado?",
    }
)

'\nAI: Desmantelar significa desmontar o romper algo. En este caso, Caperucita Roja y su abuela logran desmantelar los planes del lobo al enfrentarse a él y protegerse mutuamente.'

## Chain with chat history ([source](https://python.langchain.com/docs/use_cases/question_answering/chat_history#chain-with-chat-history))

In [26]:
qa_system_prompt = """Eres un asistente para responder preguntas. \
Utiliza la siguiente información del contexto para responder la pregunta. \
Si no sabes la respuesta, simplemente di que no la sabes. \
Intenta mantener la respuesta concisa y usar máximo tres oraciones. \
{context}
"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

def contextualized_question(input: dict):
    if input.get("chat_history"):
        return contextualize_q_chain
    else:
        return input["question"]

rag_chain = (
    RunnablePassthrough.assign(
        context=contextualized_question | retriever | format_docs
    )
    | qa_prompt
    | llm
)

In [30]:
chat_history = []

question = "Que pasa con el lobo en esta version del cuento?"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
ai_msg

'\nAsistente: En esta versión del cuento, el lobo intenta engañar a Caperucita Roja y a su abuela para comerlas. Sin embargo, la niña se cuida mucho de no hacerle caso al lobo y le cuenta a su abuela lo que pasó. La abuela cierra la puerta para protegerse del lobo, quien finalmente es derrotado en el bosque.'

In [31]:
chat_history.extend([HumanMessage(content=question), ai_msg])

second_question = "Como derrotaron al lobo?"
rag_chain.invoke({"question": second_question, "chat_history": chat_history})

'\nAsistente: El lobo fue derrotado cuando Caperucita Roja y su abuelita lo atraparon en una situación donde él intentó comer a la niña, pero cayendo accidentalmente en un pozo de agua cerca. La pequeña se llevó el crédito por salvarla a ambas, aunque fue gracias a las acciones de su abuela que pudo derrotar al lobo.'